<a href="https://colab.research.google.com/github/psy794/stock_pjtt/blob/master/LR_0713_(%EA%B2%B0%EA%B3%BC_0_40425).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
!pip install yfinance
!pip install finance-datareader
!pip install pykrx

In [2]:
import pandas as pd
import numpy as np
import random
import os

import pickle

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

#증권시장의 주식정보 스크래핑
#pip install pykrx
from pykrx import stock
from pykrx import bond

import yfinance as yf
import FinanceDataReader as fdr
import pykrx

import warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Data Load

In [47]:
# 저장된 pickle 파일 불러오기
# loaded_train = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/train_data_df_new.pkl', 'rb'))
loaded_train = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/train_data.pkl', 'rb'))

In [48]:
train = loaded_train
train

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.040000,060310
1,2021-06-02,2915,2975,2830,2900,134833,388795625,-0.680000,060310
2,2021-06-03,2900,2925,2875,2900,144470,419668300,0.000000,060310
3,2021-06-04,2930,3120,2920,2950,934224,2840373820,1.720000,060310
4,2021-06-07,3000,3150,2955,3150,946560,2929678225,6.780000,060310
...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,6710,6710,6420,6430,40981,266207710,-4.170000,238490
987996,2023-05-24,6400,6490,6210,6320,19392,123635550,-1.710000,238490
987997,2023-05-25,6340,6400,6270,6330,6457,40862130,0.160000,238490
987998,2023-05-26,6330,6410,6300,6330,8905,56621540,0.000000,238490


#지금은 새벽 한시반 7월13일



X : 2021-06-01부터 2021-06-15기간까지의 [종가,종목코드,날짜]를 제외한 컬럼
Y : 2021-06-16의 종가

반복해서,
X : 2021-06-02부터 2021-06-16기간까지의  [종가,종목코드,날짜]를 제외한 컬럼
Y : 2021-06-17의 종가

이런식으로 X값은 14일의 기간으로 1일씩 움직이고,   
y값은 1일씩 움직이는것으로해서 시계열 데이터를 먼저 만들어 둔 다음에  
전체 시계열 데이터에서 앞의 70%를 train데이터로 둬서 학습시키고 실제 종가로 결과값을 확인.  
나머지 30%데이터를 test데이터로 둬서 예측을해보고 실제 종가와 비교  

성능을 향상시킬 수 있도록하기 위해 성능평가테이블 만드는 코드를 추가하여 성능도 확인할 수 있도록 소스코드도 추가.  






In [40]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np


# 반복적으로 학습 및 예측 수행
train_start_date = pd.to_datetime('2021-06-01')
train_end_date = pd.to_datetime('2021-06-15')
test_start_date = pd.to_datetime('2021-06-16')
test_end_date = pd.to_datetime('2021-06-30')

# 변수 초기화
total_return = 0.0
num_predictions = 0

# 결과를 저장할 DataFrame 생성
results_df = pd.DataFrame(columns=['날짜', '종목코드', '절대값', '차이'])

while test_end_date <= pd.to_datetime('2023-06-21'):  # 최종 예측일까지 반복
    # 학습 데이터 설정
    train_data = train[(train['날짜'] >= train_start_date) & (train['날짜'] <= train_end_date)]
    features = train_data.drop(['종가'], axis=1)
    labels = train_data['종가']

    # 날짜 데이터를 숫자형으로 변환
    # 모델이 일자 간의 관계를 학습할 수 있고, 시계열 데이터의 패턴을 이해하고 예측할 수 있게
    features['날짜'] = features['날짜'].dt.strftime('%Y%m%d').astype(np.int32)

    # 스케일링
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)

    # 모델 학습
    model = LinearRegression()
    model.fit(features_scaled, labels)

    # 예측 데이터 설정
    test_data = train[(train['날짜'] >= test_start_date) & (train['날짜'] <= test_end_date)]
    test_features = test_data.drop(['종가'], axis=1)

    # 날짜 데이터를 숫자형으로 변환
    test_features['날짜'] = test_features['날짜'].dt.strftime('%Y%m%d').astype(np.int32)

    # 스케일링
    test_features_scaled = scaler.transform(test_features)

    # 예측
    predictions = model.predict(test_features_scaled)

    # 종목 코드별로 결과 저장
    for i, pred in enumerate(predictions):
        actual_close = test_data.iloc[i]["종가"]
        print(f'일자: {test_data.iloc[i]["날짜"]}, 종목 코드: {test_data.iloc[i]["종목코드"]}, 예측 종가: {pred}, 실제 종가: {actual_close}')
        if actual_close != 0:
            return_rate = (pred - actual_close) / actual_close
            total_return += return_rate
            num_predictions += 1

            # 결과를 DataFrame에 추가 (종목 코드별로 하나씩)
            if test_data.iloc[i]["종목코드"] not in results_df['종목코드'].values:
                results_df = results_df.append({'날짜': test_data.iloc[i]["날짜"], '종목코드': test_data.iloc[i]["종목코드"], '절대값': abs(return_rate), '차이': return_rate}, ignore_index=True)
            else:
                results_df.loc[results_df['종목코드'] == test_data.iloc[i]["종목코드"], '절대값'] += abs(return_rate)
                results_df.loc[results_df['종목코드'] == test_data.iloc[i]["종목코드"], '차이'] += return_rate

    # 다음 구간 설정
    train_start_date += pd.DateOffset(days=1)
    train_end_date += pd.DateOffset(days=1)
    test_start_date += pd.DateOffset(days=1)
    test_end_date += pd.DateOffset(days=1)

# 최종 수익률 계산
final_return = total_return / num_predictions
print("최종 수익률:", final_return)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
일자: 2022-06-29 00:00:00, 종목 코드: 228340, 예측 종가: 4006.0663242569062, 실제 종가: 3690
일자: 2022-06-30 00:00:00, 종목 코드: 228340, 예측 종가: 4017.069221827187, 실제 종가: 3675
일자: 2022-07-01 00:00:00, 종목 코드: 228340, 예측 종가: 4443.392734770903, 실제 종가: 3675
일자: 2022-06-20 00:00:00, 종목 코드: 092780, 예측 종가: 4964.362219457802, 실제 종가: 5010
일자: 2022-06-21 00:00:00, 종목 코드: 092780, 예측 종가: 5321.758501856479, 실제 종가: 5030
일자: 2022-06-22 00:00:00, 종목 코드: 092780, 예측 종가: 4645.871792257997, 실제 종가: 4640
일자: 2022-06-23 00:00:00, 종목 코드: 092780, 예측 종가: 4654.508531112238, 실제 종가: 4465
일자: 2022-06-24 00:00:00, 종목 코드: 092780, 예측 종가: 5096.728125061245, 실제 종가: 4630
일자: 2022-06-27 00:00:00, 종목 코드: 092780, 예측 종가: 5419.9207531961665, 실제 종가: 4875
일자: 2022-06-28 00:00:00, 종목 코드: 092780, 예측 종가: 5404.4572035976225, 실제 종가: 4975
일자: 2022-06-29 00:00:00, 종목 코드: 092780, 예측 종가: 5234.722456731173, 실제 종가: 4980
일자: 2022-06-30 00:00:00, 종목 코드: 092780, 예측 종가: 5197.410131508157, 실제 종가: 4875
일자: 2022-07-01 00:00:00, 

KeyboardInterrupt: ignored

In [43]:
model

LinearRegression()

In [46]:
#위의 내용에서 다른것들은 생각하 필요없다. 왜냐하면 잘못되었기때문이다.
#학습한 모델만 가져다가 예측해보자. > 아침에 눈떴는데도 안끝나서 그냥 멈췃음
#7ㅝㄹ13일아침에 눈뜨면 모델에 2023.05.14~20230530 데이터 넣어서 돌려볼것.


import joblib
# 모델 저장
joblib.dump(model, ('/content/drive/MyDrive/Colab Notebooks/model.pkl'))

# 모델 불러오기
loaded_model = joblib.load('/content/drive/MyDrive/Colab Notebooks/model.pkl')
loaded_model
# 테스트 데이터의 종가('종가' 컬럼)를 제외한 나머지 특성 데이터를 사용하여 스케일링한 값을 test_features_scaled에 두기
# 스케일링
# test_features_scaled = scaler.transform(test_features)


# # 불러온 모델로 예측 수행
# predictions = loaded_model.predict(test_features_scaled)


LinearRegression()

In [49]:
train_data = train[(train['날짜'] >= '2023-05-14') & (train['날짜'] <= '2023-05-30')]
train_data

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드
483,2023-05-15,2270,2280,2225,2240,69534,155879710,-1.750000,060310
484,2023-05-16,2240,2250,2185,2215,116835,257647860,-1.120000,060310
485,2023-05-17,2215,2260,2200,2235,97747,218697165,0.900000,060310
486,2023-05-18,2245,2290,2240,2275,133586,302686495,1.790000,060310
487,2023-05-19,2300,2310,2275,2290,242755,556024275,0.660000,060310
...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,6710,6710,6420,6430,40981,266207710,-4.170000,238490
987996,2023-05-24,6400,6490,6210,6320,19392,123635550,-1.710000,238490
987997,2023-05-25,6340,6400,6270,6330,6457,40862130,0.160000,238490
987998,2023-05-26,6330,6410,6300,6330,8905,56621540,0.000000,238490


In [50]:
train_data['날짜'] = train_data['날짜'].dt.strftime('%Y%m%d').astype(np.int32)

In [56]:
 test_data = train_data

In [57]:
test_features = test_data.drop(['종가'], axis=1)

In [58]:
test_features

,날짜,시가,고가,저가,거래량,거래대금,등락률,종목코드
483,20230515,2270,2280,2225,69534,155879710,-1.750000,060310
484,20230516,2240,2250,2185,116835,257647860,-1.120000,060310
485,20230517,2215,2260,2200,97747,218697165,0.900000,060310
486,20230518,2245,2290,2240,133586,302686495,1.790000,060310
487,20230519,2300,2310,2275,242755,556024275,0.660000,060310
...,...,...,...,...,...,...,...,...
987995,20230523,6710,6710,6420,40981,266207710,-4.170000,238490
987996,20230524,6400,6490,6210,19392,123635550,-1.710000,238490
987997,20230525,6340,6400,6270,6457,40862130,0.160000,238490
987998,20230526,6330,6410,6300,8905,56621540,0.000000,238490


In [59]:
test_features_scaled = scaler.transform(test_features)
predictions = loaded_model.predict(test_features_scaled)

In [60]:
predictions

array([62383.68999417, 62382.816643  , 62523.97659496, ...,
       66665.57995248, 66700.18040414, 69211.35890013])

In [61]:
results_df

,날짜,종목코드,절대값,차이
0,2021-06-16,060310,986.264219,822.720535
1,2021-06-16,095570,597.956041,502.824391
2,2021-06-16,006840,160.732429,120.023306
3,2021-06-16,054620,251.275623,196.284228
4,2021-06-16,265520,131.060883,104.775187
...,...,...,...,...
1995,2021-06-16,189980,825.775826,682.298986
1996,2021-06-16,000540,844.491483,699.662577
1997,2021-06-16,003280,2006.617058,151.513281
1998,2021-06-16,037440,490.435538,405.039593


In [62]:
pickle.dump(results_df, open('/content/drive/MyDrive/Colab Notebooks/만들다만피클.pkl', 'wb'))

## 그냥 빼고 더하고 다 해보기

In [66]:
results_df['절대값-차이'] = results_df['절대값'] - results_df['차이']
results_df['뺀거순위'] = results_df['절대값-차이'].rank(method='first', ascending=False).astype('int')
results_df['절대값+차이'] = results_df['절대값'] + results_df['차이']
results_df['더하거순위'] = results_df['절대값+차이'].rank(method='first', ascending=False).astype('int')
results_df

,날짜,종목코드,절대값,차이,순위,절대값-차이,뺀거순위,절대값+차이,더하거순위
811,2021-06-16,A004410,9781.346218,7367.234443,1,2414.111775,45,17148.580661,1
1625,2021-06-16,A015590,6861.253065,5708.945709,2,1152.307355,74,12570.198774,2
422,2021-06-16,A096350,6190.445923,5129.444512,3,1061.001411,80,11319.890436,4
1627,2021-06-16,A051780,6017.464784,4918.138405,4,1099.326379,75,10935.603189,5
611,2021-06-16,A021880,5932.542717,4849.222786,5,1083.319930,78,10781.765503,6
...,...,...,...,...,...,...,...,...,...
857,2021-06-16,A021820,3721.141677,-3425.480624,1996,7146.622302,5,295.661053,1431
1799,2021-06-16,A106080,3671.137083,-3476.575774,1997,7147.712857,4,194.561309,1634
855,2021-06-16,A024830,3678.719325,-3491.034353,1998,7169.753678,3,187.684973,1650
107,2021-06-16,A003620,3666.138857,-3510.850053,1999,7176.988910,2,155.288803,1711


In [67]:
pickle.dump(results_df, open('/content/drive/MyDrive/Colab Notebooks/not_yet_results_df.pkl', 'wb'))

In [76]:
selected_columns = ['종목코드', '뺀거순위']
new_df = results_df[selected_columns].copy()
new_df

,종목코드,뺀거순위
811,A004410,45
1625,A015590,74
422,A096350,80
1627,A051780,75
611,A021880,78
...,...,...
857,A021820,5
1799,A106080,4
855,A024830,3
107,A003620,2


In [77]:
new_df.sort_values('뺀거순위', ascending=True)

,종목코드,뺀거순위
1628,A115180,1
107,A003620,2
855,A024830,3
1799,A106080,4
857,A021820,5
...,...,...
465,A023450,1996
1821,A002960,1997
1812,A034950,1998
652,A134380,1999


In [71]:
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv')
sample_submission

,종목코드,순위
0,A000020,1
1,A000040,2
2,A000050,3
3,A000070,4
4,A000080,5
...,...,...
1995,A375500,1996
1996,A378850,1997
1997,A383220,1998
1998,A383310,1999


In [78]:
# baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='left')
baseline_submission = sample_submission[['종목코드']].merge(new_df[['종목코드', '뺀거순위']], on='종목코드', how='left')
baseline_submission

,종목코드,뺀거순위
0,A000020,1471
1,A000040,123
2,A000050,1396
3,A000070,1969
4,A000080,1773
...,...,...
1995,A375500,1739
1996,A378850,719
1997,A383220,480
1998,A383310,1411


In [79]:
baseline_submission.rename(columns={'뺀거순위': '순위'}, inplace=True)
baseline_submission

,종목코드,순위
0,A000020,1471
1,A000040,123
2,A000050,1396
3,A000070,1969
4,A000080,1773
...,...,...
1995,A375500,1739
1996,A378850,719
1997,A383220,480
1998,A383310,1411


In [80]:
baseline_submission.to_csv('/content/drive/MyDrive/Colab Notebooks/baseline_submission0713_빼서.csv', index=False)